In [ ]:
import duckdb as dd
from duckdb import DuckDBPyConnection
import pandas as pd
import matplotlib.pyplot as plt

### Staging Tabelle

In [ ]:
connect = dd.connect("data\\warehouse.duckdb")

In [ ]:
connect.execute(
    "CREATE OR REPLACE TABLE staging_data AS " \
    "SELECT * FROM read_csv_auto('data\\example.csv')")

In [ ]:
connect.sql("select * from staging_data limit 5")

In [ ]:
connect.sql("describe select * from staging_data limit 5")

### Erste kleine Analysen
Es geht um das Kennenlernen der Daten. Die Abfragen beantworten erste Annahmen. 

In [ ]:
# Wie viele unterschiedliche Kunden gibt es?
query = """
SELECT 
  count(DISTINCT "Customer ID") as Anzahl
FROM staging_data  
"""
connect.sql(query)

In [ ]:
# Wie oft kommen die 7 Services vor?
query = """
SELECT
  Servicename,
  count(Servicename) as Anzahl
FROM staging_data
GROUP BY Servicename
ORDER BY Anzahl DESC
"""
connect.sql(query)

In [ ]:
# Welche Services haben wir?
# einmalige Services und ihre Kosten. ROW_Number() nummeriert durch
query = """
WITH cte AS (
SELECT DISTINCT
  Servicename,
  Costs
FROM staging_data
GROUP BY Servicename, Costs
ORDER BY Costs)
SELECT 
  ROW_NUMBER() OVER () AS ID,
  Servicename, 
  Costs
FROM CTE
"""
connect.sql(query)

In [ ]:
# Die Abfrage macht nur Sinn wenn der Preis sich ändern würde. Das ist hier nicht der Fall
query = """
SELECT DISTINCT
  Servicename,
  AVG(Costs)
FROM staging_data
GROUP BY Costs, Servicename
ORDER BY Costs
"""
connect.sql(query)

In [ ]:
# Anzahl der Verkäufe pro Jahr
query = """
SELECT 
  EXTRACT(YEAR FROM "Purchase date") AS Year,
  COUNT(*) as Verkäufe
FROM staging_data
GROUP BY Year
ORDER BY Year
"""
connect.sql(query)

In [ ]:
# Verteilung auf Monate
query = """
SELECT
  STRFTIME("Purchase date", '%Y-%m') AS Jahr_Monat,
  COUNT(*) as Verkäufe
FROM staging_data
GROUP BY Jahr_Monat
ORDER BY Verkäufe DESC
LIMIT 10
"""
connect.sql(query)

In [ ]:
# Verteilung auf Wochen
query = """
SELECT
  STRFTIME("Purchase date", '%Y-%W') AS Jahr_Woche,
  COUNT(*) as Verkäufe
FROM staging_data
GROUP BY Jahr_Woche
ORDER BY Verkäufe DESC
LIMIT 10
"""
connect.sql(query)

### Dimensionstabellen erstellen

In [ ]:
# Services
query = """
CREATE OR REPLACE TABLE tblServices AS
WITH cte AS (
SELECT DISTINCT
  Servicename,
  Costs
FROM staging_data
ORDER BY Costs)
SELECT 
  ROW_NUMBER() OVER () AS service_id,
  Servicename, 
  Costs
FROM CTE
"""
connect.sql(query)

In [ ]:
# Customers
query = """
CREATE OR REPLACE TABLE tblCustomers AS
SELECT DISTINCT
  "Customer ID" as customer_id,
  First_Name,
  Last_Name,
  Gender,
  City,
  "Support Level" as support_level,
  Birthday,
  date_diff('year', Birthday, today()) as Age
FROM staging_data
ORDER BY Last_Name
"""
connect.sql(query)

In [ ]:
connect.sql("select * from tblCustomers where customer_id = '3114143559'")

In [ ]:
query = """
CREATE OR REPLACE TABLE tblOrders AS
SELECT 
  ROW_NUMBER () OVER () as order_id,
  sd."Purchase date" as purchase_date,
  c.customer_id,
  s.service_id,
  sd.payment_method,
  sd."Sales Canal" as sales_canal,
  sd."Customer Satisfaction" as satisfaction
FROM 
  staging_data sd
JOIN tblCustomers c ON sd."Customer ID" = c.customer_id
JOIN tblServices s ON sd.Servicename = s.Servicename
"""
connect.sql(query)

### Tabellen überprüfen

In [ ]:
connect.sql("select * from tblOrders limit 5")

In [ ]:
# Alle Daten wieder zusammensetzen. 
query = """
SELECT 
  purchase_date,
  first_name,
  last_name,
  gender,
  support_level,
  city,
  servicename,
  payment_method,
  satisfaction
FROM
  tblOrders o
JOIN tblCustomers c ON o.customer_id = c.customer_id
JOIN tblServices s ON s.service_id = o.service_id
LIMIT 10
"""
connect.sql(query)

In [ ]:
connect.sql("SELECT COUNT(DISTINCT customer_id) FROM tblCustomers;")

In [ ]:
connect.sql("SELECT COUNT(DISTINCT servicename) FROM tblServices;")

In [ ]:
# haben wir Customer doppelt in der tblCustomers?
query = """
  SELECT
    customer_id,
    COUNT(*) as counter
  FROM tblCustomers c
  GROUP BY c.customer_id
  HAVING counter > 1
"""
connect.sql(query)

In [ ]:
connect.sql("SELECT COUNT(DISTINCT customer_id) FROM tblCustomers;")


In [ ]:
connect.sql("SELECT COUNT(DISTINCT service_id) FROM tblServices;")

In [ ]:
connect.sql("SELECT COUNT(DISTINCT order_id) FROM tblOrders;")

### Analyse
Hier werden die Daten ausgewertet. 

In [ ]:
# Wie viele Bestellungen gibt es insgesamt?
query = """
SELECT 
  COUNT(*) as total_orders
FROM tblOrders
"""
connect.sql(query)

In [ ]:
# Welches Jahr war das Umsatzstärkste?
query = """
SELECT 
  SUM(s.costs) as in_total,
  count(o.purchase_date) as in_total_orders,
  extract('year' FROM o.purchase_date) as year
FROM 
  tblOrders o
JOIN tblServices s ON o.service_id = s.service_id
GROUP BY year
ORDER BY in_total desc
"""
connect.sql(query)


### Das stärkste Jahr grafisch

In [ ]:
query = """
SELECT 
  SUM(s.costs) as in_total,
  extract('year' FROM o.purchase_date) as year
FROM 
  tblOrders o
JOIN tblServices s ON o.service_id = s.service_id
GROUP BY year
ORDER BY in_total desc
LIMIT 5
"""
df = connect.execute(query).df()
plt.plot(df['in_total'], df['year'])

In [ ]:
# Welcher Service ist der beliebteste?
query = """
SELECT 
  count(s.service_id) as in_total,
  s.Servicename
FROM 
  tblOrders o
JOIN tblServices s ON o.service_id = s.service_id
GROUP BY o.service_id, s.Servicename
ORDER BY in_total desc
"""
connect.sql(query)

### Der beliebteste Service grafisch

In [ ]:
query = """
SELECT 
  count(s.service_id) as in_total,
  s.Servicename
FROM 
  tblOrders o
JOIN tblServices s ON o.service_id = s.service_id
GROUP BY o.service_id, s.Servicename
ORDER BY in_total desc
"""
df = connect.query(query).df()
plt.xticks(rotation=45)
plt.plot(df['Servicename'], df['in_total'])

In [ ]:
# Welcher Service ist der Beliebteste aber mit Window Functions. 
query = """
SELECT DISTINCT
  s.Servicename
  ,COUNT() OVER (PARTITION BY s.Servicename) as Counter
FROM tblOrders o
JOIN tblServices s ON o.service_id = s.service_id
ORDER BY Counter desc
LIMIT 7
"""
connect.sql(query)

### Kundenverhalten

In [ ]:
# Wer sind die Top 5 Kunden?
query = """
SELECT
  customer_id,
  count(*) as order_count
FROM tblOrders
GROUP BY customer_id
ORDER BY order_count desc
LIMIT 5
"""
connect.sql(query)

### Fürs Dashboard

In [ ]:
# Anzahl der Bestellungen einer Stadt
def get_city_profit(city: str, con: DuckDBPyConnection) -> pd.DataFrame:
    query = """
    SELECT 
      s.servicename,
      count(s.servicename) as counter,
      sum(s.costs) as total_costs
    FROM 
      tblOrders o
    JOIN tblCustomers c ON o.customer_id = c.customer_id
    JOIN tblServices s ON o.service_id = s.service_id
    WHERE c.City = $city
    GROUP BY s.servicename
    ORDER BY total_costs DESC
    """
    result = con.execute(query, {'city': city}).df()
    return result


In [ ]:
get_city_profit('Tylerberg', connect)

In [63]:
connect.close()